## Red Neuronal Convolucional

La arquitectura de esta red será de dos capas convolucionales, después de la primera se hace una operación de Pooling (MaxPool aunque se puede modificar por AvgPool, al igual que el tamaño del movimiento). Finalmente se pasará a una porción totalmente conexa de 4 capas. La función de activación será RELU para todas las capas. No se agregan valores de dropout.

In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [46]:
class CNN(nn.Module):

    #En el constructor definimos la arquitectura
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,6,5) 
        self.pool1 = nn.MaxPool2d(2,2) #Filtro MaxPool de tamaño 2x2, recorre bloques unitarios
        self.conv2 = nn.Conv2d(6,16,5)
        
        #self.pool2 = nn.AvgPool2d(3, stride=2) #Filtro AvgPool de tamaño 3x3, recorriendo bloques de dos en dos
        # Esta capa se puede agregar pero se tienen que recalcular los valores de salida.
        
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120, 100)
        self.fc3 = nn.Linear(100, 69)
        self.fc4 = nn.Linear(69, 20)
        self.fc5 = nn.Linear(20, 10) #Es una red para clasificación de 10 clases (ahorita veremos el dataset en cuestión)

    
    #En esta función definimos las funciones de activación entre cada capa, todas son RELU
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool1(F.relu(self.conv2(x)))
        #x = self.pool2(F.relu(self.conv2(x)))
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

# Inicializamos la red
red_cnn = CNN()

In [47]:
red_cnn

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=69, bias=True)
  (fc4): Linear(in_features=69, out_features=20, bias=True)
  (fc5): Linear(in_features=20, out_features=10, bias=True)
)

### Funciones de pérdida

Al tener un dataset donde hay 10 clases para clasificar, usaremos CrossEntropy (CrossEntropyLoss) como función de pérdida. De igual manera usaremos el optimizador adam con una razón de aprendizaje de 0.003

In [48]:
loss_fun = nn.CrossEntropyLoss()
optim1 = optim.SGD(red_cnn.parameters(), lr=0.0001, momentum = 0.9)
#optimizador = optim.Adam(red_cnn.parameters(), lr = 0.003, betas = (0.9,0.999), eps= 1e-08)

### Entrenamiento

Antes de implementar el entrenamiento tenemos que tener información de entrenamiento, usaremos datasets extraídos de pytorch mismo. Para el entrenamiento haremos una iteración en 4 épocas siguiendo los lineamientos de la documentación de pytorch.

In [49]:
# Cargamos el conjunto de datos

import torch
import torchvision
import torchvision.transforms as transforms

# Importamos un dataset de prueba para poder entrenar el conjunto de datos.

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
for epoch in range(2):
    loss_total = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optim1.zero_grad()
        #optimizador.zero_grad()

        outputs = red_cnn(inputs)
        loss = loss_fun(outputs, labels)
        loss.backward()
        optimizador.step()

        loss_total += loss.item()
        if i % 2000 == 1999:
            print(f'[{epoch+1}, {i+1:5d}] loss: {loss/2000:.3f}')
print("Hemos acabado")

[1,  2000] loss: 0.001
[1,  4000] loss: 0.001
[1,  6000] loss: 0.001
